In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

PATH_TO_DATA = 'http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv'
data = pd.read_csv(PATH_TO_DATA, header=None)

X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

input_dim = X_train.shape[1]

encoder_input = tf.keras.Input(shape=(input_dim,))
x = tf.keras.layers.Dense(64, activation='relu')(encoder_input)
x = tf.keras.layers.Dense(32, activation='relu')(x)
latent = tf.keras.layers.Dense(16, activation='relu')(x)
encoder = tf.keras.Model(encoder_input, latent, name="encoder")

decoder_input = tf.keras.Input(shape=(16,))
x = tf.keras.layers.Dense(32, activation='relu')(decoder_input)
x = tf.keras.layers.Dense(64, activation='relu')(x)
decoder_output = tf.keras.layers.Dense(input_dim, activation='sigmoid')(x)
decoder = tf.keras.Model(decoder_input, decoder_output, name="decoder")

autoencoder_input = tf.keras.Input(shape=(input_dim,))
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)
autoencoder = tf.keras.Model(autoencoder_input, decoded, name="autoencoder")

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
autoencoder.compile(optimizer=optimizer, loss='mse')

autoencoder.fit(X_train, X_train, epochs=20, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

encoded_X_test = encoder.predict(X_test)

reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

threshold = np.mean(mse) + 3 * np.std(mse)

anomalies = np.where(mse > threshold)[0]

print("Indices of anomalies:", anomalies)

2024-11-08 08:08:59.542324: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 08:08:59.669334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 08:09:00.828071: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 08:09:01.521776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731033542.174792   39432 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731033542.33